In [1]:
import itertools

In [2]:
N = ToricLattice(7)
K = N([-3, 1, 1, 1, 1, 1, 1]) # canonical divisor
E = identity_matrix(QQ, 7)[:,1:].columns() # E is the set of exceptional curves, so the basis is L, E[0], .. , E[5]
E = [N(e) for e in E]
Line = lambda exceptional_curves : N([i/3 for i in (-K + sum(exceptional_curves)) ])
L = Line(E)

Q = diagonal_matrix([1]+6*[-1])
dot = lambda a, b: a*Q*b
gram_matrix = lambda rays: matrix([[dot(a,b) for a in rays] for b in rays])

minus_one_curves = E + [L-ei-ej for ei,ej in itertools.combinations(E, 2)] + [-K-L+e for e in E]
curves_disjoint_with = lambda curves: [c for c in minus_one_curves if all(dot(c,c2)==0 for c2 in curves)]

NE = Cone(minus_one_curves)
Ample = Cone([N(Q*ray) for ray in NE.dual().rays()])

# take one Fujita face for each isomorphism class (determined by CPW, may check explicitly later)
# and turn them into subcones in Mori cone by adding -K

NE_face_types_rays = {
        "B6": E,
        "B5": E[:5],
        "B5'": E[:4]+[L-E[4]-E[5]], # B(5) for P1xP1. Another type of independent set of 5 (-1)-curves, this one is maximal
        "B4": E[:4],
        "B3": E[:3],
        "B2": E[:2],
        "B1": E[:1],
        #'C6': E[:5]+[L-E[5]-e for e in E[:5]], # C(6). Schlafli graph is 4-ultrahomogeneous, and, after fixing two pairs, three others are defined uniquely
        "C5": E[:5]+[L-E[5]], # здесь и ниже - части C6, из подобных которым складывается весь С6 
        "C5'": E[:4]+[L-E[5]-E[4], L-E[5]],
        "C4": E[:4]+[L-E[5]],
        "C3": E[:3]+[L-E[5]],
        "C2": E[:2]+[L-E[5]],
        "C1": E[:1]+[L-E[5]],
        "C0": E[:0]+[L-E[5]],
    }
cone_of_type = {label:Cone(rays+[-K]).intersection(Ample) for label,rays in NE_face_types_rays.items()}

# returns True if rel.int.(cone1) is in rel.int.(cone2)
is_covered_by = lambda cone1, cone2: all(cone2.contains(ray) for ray in cone1.rays()) and cone2.interior_contains(sum(cone1.rays()))

In [3]:
# Constructing cylinders.

def independent_sets(vectors, size = 6):
    if size == 0:
        yield []
    for i, v in enumerate(vectors):
        orthogonals = [v2 for v2 in vectors[i+1:] if dot(v, v2)==0]
        for subset in independent_sets(orthogonals, size-1):
            yield subset + [v]

#cylinder1 = lambda E : E + [Line(E)-E[-1], -K-Line(E)+E[-1]]   
def cylinder_generator_P2(E):
    '''
    E is a list of exceptional curves, and e is one of them
    returns 
    1. list of rays of the cone of divisors H such that the cylinder (P^2 minus conic through E-e and tangent line through e) is H-polar
    2. list of (-1)-curves lying in the complement of the cylinder (or a union of cylinders if many)
    '''
    for e in E:
        tangent = Line(E) - e
        conic = - K - tangent
        curves = E+[conic]
        yield curves + [tangent]

def cylinder_generator_1(E):
    """
    cylinder complement to E and lines through blowdowns of e and one of E-e
    """
    for e in E:
        curves = E+[Line(E)-e-f for f in E if f!=e]
        yield curves

def cylinder_generator_P1(E):
    '''
    CPW Example 4.1.6 and Lemma 4.2.2 for contraction of E1..E4, L-E5-E6.
    '''
    for i,j in itertools.permutations(range(len(E)),int(2)):
        complement = [k for k in range(len(E)) if (k!=i) and (k!=j)]
        conic = -K-Line(E)+E[i]
        curves = [E[k] for k in complement]+[Line(E)-E[i]-E[j], conic]
        yield curves + [Line(E)-E[i],Line(E)-E[j]]

def cylinders_of_type(cylinder_generator_list):
    '''
    returns a list of cones of cylinders of considered type
    '''
    for cylinder_generator in cylinder_generator_list:
        for blowdown in independent_sets(minus_one_curves):
            yield from cylinder_generator(blowdown)

# polarity cone of a collection of cylinders
def Pol(collection):
    result = Cone([],lattice=N.dual()).dual() # ambient space
    for cylinder in collection:
        result = result.intersection(Cone(cylinder['rays']))
    return result

def minus_one_curves_in_complement(collection):
    return [curve for curve in minus_one_curves if all(curve in cylinder['rays'] for cylinder in collection)]
    
# forbidden cone of a collection of cylinders
def Forb(collection):
    return Cone(minus_one_curves_in_complement(collection),lattice=N)

cylinders = [{
    'rays': cylinder_rays,
    'index': i,
} for i, cylinder_rays in enumerate(cylinders_of_type([cylinder_generator_P2,cylinder_generator_P1]))]

for u in cylinders:
    u['Pol'] = Pol([u])
    u['Forb'] = Forb([u])


In [66]:

def is_polar_wrt(cylinder, cones):
    return all(is_covered_by(cone,cylinder['Pol']) for cone in cones)


N(0,  1,  0,  0,  0,  0, 0),
N(0,  0,  1,  0,  0,  0, 0),
N(0,  0,  0,  1,  0,  0, 0),
N(0,  0,  0,  0,  1,  0, 0),
N(0,  0,  0,  0,  0,  1, 0),
N(0,  0,  0,  0,  0,  0, 1),
N(2, -1, -1, -1, -1, -1, 0)
in 7-d lattice N

In [146]:
cone_types_list_1 = [cone_of_type[i] for i in ["B6","B5","B4","B3","B2","B1",]]

U=[c for c in cylinders if is_polar_wrt(c,cone_types_list_1)]
[u['index'] for u in U]
#u_1=cylinders[5]
#Forb(U).intersection(Ample)

[5, 11, 17, 23, 29, 35, 41, 47, 53, 59, 65, 71, 77, 83, 89, 95]

In [152]:
cone_types_list_2 = [cone_of_type[i] for i in ["C5",]]

U=[c for c in cylinders if is_polar_wrt(c,cone_types_list_2)]
[u['index'] for u in U]


[0]

In [153]:
cone_types_list_7 = [cone_of_type[i] for i in ["C3","C2","C1","C0",]]

U=[c for c in cylinders if is_polar_wrt(c,cone_types_list_7)]
[u['index'] for u in U]


[437, 442, 463, 468, 498, 528, 588, 618, 918, 948]

In [102]:
cone_types_list_3 = [cone_of_type[i] for i in ["B5'","C5'",]]

U=[c for c in cylinders if is_polar_wrt(c,cone_types_list_3)]
[u['index'] for u in U]
#Forb([cylinders[437],cylinders[432]]).rays()


[432, 437, 468, 473, 498, 503, 588, 593, 918, 923]

In [122]:
cone_types_list_4 = [cone_of_type[i] for i in ["C4",]]

U=[c for c in cylinders if is_polar_wrt(c,cone_types_list_4)]
[u['index'] for u in U]
#U4 = [cylinders[437]]
#Forb(U4).rays()


[437, 468, 498, 588, 918]

In [119]:
cone_of_type["C4'"] = cylinders[437]['Forb'].intersection(cone_of_type['C4'])
U=[c for c in cylinders if is_polar_wrt(c,[cone_of_type["C4'"]])]
[u['index'] for u in U]
U5=[c for c in cylinders if c['index'] in [437,442]]
Forb(U5).intersection(Ample)

0-d cone in 7-d lattice N

In [183]:
cone_types_list_5 = [cone_of_type[i] for i in ["C4'","C3","C2","C1","C0",]]
print(Forb(U3).rays())
print(NE_face_types_rays["C3"])


N(0,  1,  0,  0,  0,  0,  0),
N(0,  0,  1,  0,  0,  0,  0),
N(0,  0,  0,  1,  0,  0,  0),
N(0,  0,  0,  0,  1,  0,  0),
N(1,  0,  0,  0,  0, -1, -1),
N(2, -1, -1, -1, -1,  0, -1)
in 7-d lattice N
[N(0, 1, 0, 0, 0, 0, 0), N(0, 0, 1, 0, 0, 0, 0), N(0, 0, 0, 1, 0, 0, 0), N(1, 0, 0, 0, 0, 0, -1)]


In [128]:
for i in  [432,437,442]:
    print(i)
    for ray in cylinders[i]['rays']:
        print(ray)

    

432
N(0, 0, 0, 0, 1, 0, 0)
N(0, 0, 0, 1, 0, 0, 0)
N(0, 0, 1, 0, 0, 0, 0)
N(0, 1, 0, 0, 0, 0, 0)
N(1, 0, 0, 0, 0, -1, -1)
N(2, -1, -1, -1, -1, -1, 0)
N(1, 0, 0, 0, 0, 0, -1)
N(1, 0, 0, 0, 0, -1, 0)
437
N(0, 0, 0, 0, 1, 0, 0)
N(0, 0, 0, 1, 0, 0, 0)
N(0, 0, 1, 0, 0, 0, 0)
N(0, 1, 0, 0, 0, 0, 0)
N(1, 0, 0, 0, 0, -1, -1)
N(2, -1, -1, -1, -1, 0, -1)
N(1, 0, 0, 0, 0, -1, 0)
N(1, 0, 0, 0, 0, 0, -1)
442
N(0, 0, 0, 0, 0, 1, 0)
N(0, 0, 0, 1, 0, 0, 0)
N(0, 0, 1, 0, 0, 0, 0)
N(0, 1, 0, 0, 0, 0, 0)
N(1, 0, 0, 0, -1, 0, -1)
N(2, -1, -1, -1, 0, -1, -1)
N(1, 0, 0, 0, -1, 0, 0)
N(1, 0, 0, 0, 0, 0, -1)


In [184]:
U1=[c for c in cylinders if c['index'] in [5]]
U2=[c for c in cylinders if c['index'] in [0]]
U3=[c for c in cylinders if c['index'] in [437]]
U4=[c for c in cylinders if c['index'] in [437,432]]
U5=[c for c in cylinders if c['index'] in [437,442]]
for key, cone in cone_of_type.items():
    for i,U in enumerate([U1,U2,U3,U4,U5]):
        print(i,key,is_covered_by(cone,Pol(U)))

0 B6 True
1 B6 True
2 B6 False
3 B6 False
4 B6 False
0 B5 True
1 B5 True
2 B5 False
3 B5 False
4 B5 False
0 B5' False
1 B5' False
2 B5' True
3 B5' True
4 B5' False
0 B4 True
1 B4 False
2 B4 False
3 B4 False
4 B4 False
0 B3 True
1 B3 False
2 B3 False
3 B3 False
4 B3 False
0 B2 True
1 B2 False
2 B2 False
3 B2 False
4 B2 False
0 B1 True
1 B1 False
2 B1 False
3 B1 False
4 B1 False
0 C5 False
1 C5 True
2 C5 False
3 C5 False
4 C5 False
0 C5' False
1 C5' False
2 C5' True
3 C5' True
4 C5' False
0 C4 False
1 C4 False
2 C4 True
3 C4 False
4 C4 False
0 C3 False
1 C3 False
2 C3 True
3 C3 False
4 C3 True
0 C2 False
1 C2 False
2 C2 True
3 C2 False
4 C2 True
0 C1 False
1 C1 False
2 C1 True
3 C1 False
4 C1 True
0 C0 False
1 C0 False
2 C0 True
3 C0 False
4 C0 True
0 C4' False
1 C4' False
2 C4' True
3 C4' False
4 C4' True


In [159]:
is_covered_by(cone_of_type["B5'"],Pol(U3))
Pol(U3).contains(E[4])

False

N(3,  0,  0,  0,  0, -1, -1),
N(3, -1, -1,  0, -1, -1, -1),
N(3,  0,  0, -1, -1, -1, -1),
N(3,  0, -1, -1, -1, -1, -1),
N(3, -1, -1, -1, -1, -1, -1),
N(3, -1,  0, -1, -1, -1, -1),
N(3, -1,  0,  0, -1, -1, -1),
N(3,  0,  0,  0, -1, -1, -1),
N(3,  0, -1,  0, -1, -1, -1),
N(3,  0, -1,  0,  0, -1, -1),
N(3, -1, -1,  0,  0, -1, -1),
N(3,  0,  0, -1,  0, -1, -1),
N(3,  0, -1, -1,  0, -1, -1),
N(3, -1, -1, -1,  0, -1, -1),
N(3, -1,  0, -1,  0, -1, -1),
N(3, -1,  0,  0,  0, -1, -1),
N(4, -1,  0,  0,  0, -2, -2),
N(2,  0,  0,  0,  0, -1, -1),
N(4, -1, -1,  0, -1, -2, -2),
N(4,  0,  0, -1, -1, -2, -2),
N(4,  0, -1, -1, -1, -2, -2),
N(4, -1, -1, -1, -1, -2, -2),
N(4, -1,  0, -1, -1, -2, -2),
N(4, -1,  0,  0, -1, -2, -2),
N(4,  0,  0,  0, -1, -2, -2),
N(4,  0, -1,  0, -1, -2, -2),
N(4,  0, -1,  0,  0, -2, -2),
N(4, -1, -1,  0,  0, -2, -2),
N(4,  0,  0, -1,  0, -2, -2),
N(4,  0, -1, -1,  0, -2, -2),
N(4, -1, -1, -1,  0, -2, -2),
N(4, -1,  0, -1,  0, -2, -2)
in 7-d lattice N

In [186]:
G=Forb(U3).intersection(Cone(NE_face_types_rays["C5'"]+[-K]))
G.dual().lines()

M(1, 0, 0, 0, 0, -1, 2)
in 7-d lattice M

In [166]:
for ray in NE_face_types_rays["C5'"]+[-K]:
    if ray not in Pol(U3):
        print(ray)

In [191]:
NE_face_types_rays["B5'"]
dot(L+E[4]-2*E[5],2*(L-E[5])-K)

0

In [193]:
[c for c in minus_one_curves if all(dot(c,d)<=0 for d in Forb(U4).rays())]

[N(0, 1, 0, 0, 0, 0, 0),
 N(0, 0, 1, 0, 0, 0, 0),
 N(0, 0, 0, 1, 0, 0, 0),
 N(0, 0, 0, 0, 1, 0, 0),
 N(1, 0, 0, 0, 0, -1, -1)]